# Лабораторна Робота № 7
## Передавальне навчання з великими мовними моделями


## Мета: набути навички використання та донавчання великих мовних моделей на базі сервісу HuggingFace для розв'язання NLP задач.

### Імпорт бібліотек та датасету

In [6]:
!pip install nltk

  Obtaining dependency information for nltk from https://files.pythonhosted.org/packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.5 MB 653.6 kB/s eta 0:00:03
   - -------------------------------------- 0.1/1.5 MB 825.8 kB/s eta 0:00:02
   - -------------------------------------- 0.1/1.5 MB 825.8 kB/s eta 0:00:02
   - -------------------------------------- 0.1/1.5 MB 825.8 kB/s eta 0:00:02
   - -------------------------------------- 0.1/1.5 MB 825.8 kB/s eta 0:00:02
   - -------------------------------------- 0.1/1.5 MB 825.8 kB/s eta 0:00:02
   - -


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np
import time
from transformers import AutoTokenizer, BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import matplotlib.pyplot as plt
import re
import nltk
from keras import models
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, f1_score
import seaborn as sns

In [ ]:
df_train = pd.read_csv("lab06_train.txt", sep=";", header=None, names=['Text', 'Emotion'])
df_test = pd.read_csv("lab06_test.txt", sep=";", header=None, names=['Text', 'Emotion'])
df_val = pd.read_csv("lab06_val.txt", sep=";", header=None, names=['Text', 'Emotion'])

In [ ]:
emotions = df_train['Emotion'].unique().tolist()

In [ ]:
df_train['Emotion'].value_counts()

Emotion
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

### Завантаження моделі, токенизатора та токенизація даних

In [ ]:
model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=len(emotions))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm
tqdm.pandas()

def tokenize_to_tf(x:str|list[str]) -> pd.Series:
  result = tokenizer(
      x,
      padding = 'max_length',
      truncation=True
  )
  return pd.Series({'input_ids':result['input_ids'], 'attention_mask':result['attention_mask']})

tokenized_text_train = df_train['Text'].progress_apply(tokenize_to_tf)
tokenized_text_test = df_test['Text'].progress_apply(tokenize_to_tf)
tokenized_text_val = df_val['Text'].progress_apply(tokenize_to_tf)

100%|██████████| 2000/2000 [00:01<00:00, 1428.95it/s]


In [ ]:
y_train = df_train['Emotion']
y_test = df_test['Emotion']
y_val = df_val['Emotion']

label_encoder = LabelEncoder()
label_encoder.fit(emotions)

y_train = tf.keras.utils.to_categorical(label_encoder.transform(y_train), num_classes=len(emotions))
y_test = tf.keras.utils.to_categorical(label_encoder.transform(y_test), num_classes=len(emotions))
y_val = tf.keras.utils.to_categorical(label_encoder.transform(y_val), num_classes=len(emotions))

In [ ]:
emotions

['sadness', 'anger', 'love', 'surprise', 'fear', 'joy']

In [ ]:
df_train[:15]

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
5,ive been feeling a little burdened lately wasn...,sadness
6,ive been taking or milligrams or times recomme...,surprise
7,i feel as confused about life as a teenager or...,fear
8,i have been with petronas for years i feel tha...,joy
9,i feel romantic too,love


In [ ]:
y_train[:15]

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
emotions = ['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((
    {'input_ids': tokenized_text_train['input_ids'].tolist(),
     'attention_mask': tokenized_text_train['attention_mask'].tolist()},
    y_train
)).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((
    {'input_ids': tokenized_text_test['input_ids'].tolist(),
     'attention_mask': tokenized_text_test['attention_mask'].tolist()},
    y_test
)).batch(32)

val_ds = tf.data.Dataset.from_tensor_slices((
    {'input_ids': tokenized_text_val['input_ids'].tolist(),
     'attention_mask': tokenized_text_val['attention_mask'].tolist()},
    y_val
)).batch(32)

### Функції графіків

In [ ]:
def plot_learning_curve(history, figsize=(10, 5)):
    fig, axs = plt.subplots(1, 2, figsize=figsize)

    axs[0].plot(history.history['accuracy'], color='#b8113d')
    axs[0].plot(history.history['val_accuracy'], color='#6010b5')
    axs[0].set_title('Model accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].legend(['Train', 'Validation'], loc='upper left')

    axs[1].plot(history.history['loss'], color='#b8113d')
    axs[1].plot(history.history['val_loss'], color='#6010b5')
    axs[1].set_title('Model loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].legend(['Train', 'Validation'], loc='upper left')

    plt.show()

In [ ]:
def plot_confusion_matrix(y_test, y_pred):
  cm = confusion_matrix(y_test, y_pred)

  plt.figure(figsize=(10, 8))
  sns.heatmap(cm, annot=True, fmt='d', cmap='PuRd', xticklabels=emotions, yticklabels=emotions)
  plt.xlabel('Predicted labels')
  plt.ylabel('True labels')
  plt.title('Confusion Matrix')
  plt.show()

### Model fine-tuning

In [ ]:
model.compile(
    optimizer = 'adam',
    metrics=['accuracy']
    )

start_time = time.time()

history = model.fit(
    train_ds,
    epochs = 3,
    batch_size = 8,
    validation_data = val_ds
)

end_time = time.time()

Epoch 1/3


ResourceExhaustedError: Graph execution error:

Detected at node tf_bert_for_sequence_classification/bert/encoder/layer_._5/attention/self/MatMul defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/iolo

### Ананліз результатів

In [ ]:
plot_learning_curve(history)

In [ ]:
pred_logits = model.predict(test_ds).logits
y_pred = tf.argmax(pred_logits).numpy()

#y_pred_uint8 = np.argmax(y_pred, axis=1).astype(np.uint8)
#y_test_unit8 = np.argmax(y_test, axis=1).astype(np.uint8)

print("Classification Report:")
print(classification_report(y_test, y_pred))

plot_confusion_matrix(y_test, y_pred)

In [ ]:
train_time = start_time - end_time

print("Fine-tuning time of the model: {:.2f} seconds".format(-train_time))